In [ ]:
# import tensorflow as tf
# from tensorflow import keras
# from tensorflow.keras import datasets, layers, models
# from tensorflow.keras.callbacks import EarlyStopping
# from tensorflow.keras.callbacks import ModelCheckpoint
# from tensorflow.keras.models import Sequential, Model
# from tensorflow.keras.layers import BatchNormalization, Conv2D, MaxPooling2D, Flatten, Dense, AveragePooling2D
# from tensorflow.keras.layers import ZeroPadding2D, add, Input, Activation
# from tensorflow.keras.regularizers import l2
from sklearn.preprocessing import LabelBinarizer
# from tensorflow.keras.optimizers import SGD
# from tensorflow.keras.optimizers import Adam , Adadelta, RMSprop, Nadam
from tensorflow.keras.datasets import cifar10
# from tensorflow.keras import backend as K
# from tensorflow.keras.callbacks import LearningRateScheduler
import matplotlib.pyplot as plt
import numpy as np
import wandb
import mxnet as mx
from wandb.keras import WandbCallback

In [ ]:
((trainX, trainY), (testX, testY)) = cifar10.load_data()

trainX = trainX.astype("float") / 255.0
testX = testX.astype("float") / 255.0

In [ ]:
lb = LabelBinarizer()
trainY = lb.fit_transform(trainY)
testY = lb.transform(testY)

labelNames = ["airplane", "automobile", "bird", "cat", "deer",
"dog", "frog", "horse", "ship", "truck"]

In [ ]:
def squeeze(input, numFilter):
# the first part of a FIRE module consists of a number of 1x1
# filter squeezes on the input data followed by an activation
    squeeze_1x1 = mx.sym.Convolution(data=input, kernel=(1, 1),
    stride=(1, 1), num_filter=numFilter)
    act_1x1 = mx.sym.LeakyReLU(data=squeeze_1x1,
    act_type="elu")
    return act_1x1

In [ ]:
def fire(input, numSqueezeFilter, numExpandFilter):
    squeeze_1x1 = squeeze(input, numSqueezeFilter)
    expand_1x1 = mx.sym.Convolution(data=squeeze_1x1,
                kernel=(1, 1), stride=(1, 1), num_filter=numExpandFilter)
    relu_expand_1x1 = mx.sym.LeakyReLU(data=expand_1x1,
                act_type="elu")
    expand_3x3 = mx.sym.Convolution(data=squeeze_1x1, pad=(1, 1),
                kernel=(3, 3), stride=(1, 1), num_filter=numExpandFilter)
    relu_expand_3x3 = mx.sym.LeakyReLU(data=expand_3x3,
                act_type="elu")
    output = mx.sym.Concat(relu_expand_1x1, relu_expand_3x3,
                dim=1)
    return output

In [ ]:
def build(classes):
     # data input
    data = mx.sym.Variable("data")

    # Block #1: CONV => RELU => POOL
    conv_1 = mx.sym.Convolution(data=data, kernel=(2, 2),
        stride=(1, 1), num_filter=64)
    relu_1 = mx.sym.LeakyReLU(data=conv_1, act_type="elu")
    pool_1 = mx.sym.Pooling(data=relu_1, kernel=(3, 3),
        stride=(1, 1), pool_type="max")

    # Block #2-4: (FIRE * 3) => POOL
    fire_2 = fire(pool_1, numSqueezeFilter=16,
        numExpandFilter=32)
    fire_3 = fire(fire_2, numSqueezeFilter=16,
        numExpandFilter=32)
    fire_4 = fire(fire_3, numSqueezeFilter=32,
         numExpandFilter=64)
    pool_4 = mx.sym.Pooling(data=fire_4, kernel=(3, 3),
        stride=(2, 2), pool_type="max")

    # Block #5-8: (FIRE * 4) => POOL
    fire_5 = fire(pool_4, numSqueezeFilter=32,
        numExpandFilter=128)
    fire_6 = fire(fire_5, numSqueezeFilter=48,
        numExpandFilter=192)
    fire_7 = fire(fire_6, numSqueezeFilter=48,
        numExpandFilter=192)
    fire_8 = fire(fire_7, numSqueezeFilter=64,
        numExpandFilter=256)
    pool_8 = mx.sym.Pooling(data=fire_8, kernel=(3, 3),
        stride=(1, 1), pool_type="max")

            # Block #9-10: FIRE => DROPOUT => CONV => RELU => POOL
    fire_9 = fire(pool_8, numSqueezeFilter=32,
        numExpandFilter=128)
    do_9 = mx.sym.Dropout(data=fire_9, p=0.5)
    conv_10 = mx.sym.Convolution(data=do_9, num_filter=classes,
        kernel=(1, 1), stride=(1, 1))
    relu_10 = mx.sym.LeakyReLU(data=conv_10, act_type="elu")
    pool_10 = mx.sym.Pooling(data=relu_10, kernel=(12, 12),
        pool_type="avg"
            # softmax classifier
    flatten = mx.sym.Flatten(data=pool_10)
    model = mx.sym.SoftmaxOutput(data=flatten, name="softmax")

    # return the network architecture
    return model

In [ ]:
BATCH_SIZE = 256

trainIter = mx.io.ImageRecordIter( path_imgrec=(trainX, trainY), data_shape=(227, 227,3)
                                  , batch_size=batchSize, rand_crop=True,
                                  rand_mirror=True, rotate=15, max_shear_ratio=0.1)

valIter = mx.io.ImageRecordIter(path_imgrec=(testX, testY), data_shape=(227, 227,3),
                                batch_size=batchSize)

metrics = [mx.metric.Accuracy()]

In [ ]:
opt = mx.optimizer.SGD(learning_rate=1e-2, momentum=0.9, wd=0.0002,
rescale_grad=1.0 / batchSize)

model = build(10)
model = model.symbol

In [ ]:
model = mx.model.FeedForward(ctx=[mx.gpu(0)],symbol=model,
initializer=mx.initializer.Xavier(),arg_params=argParams,
aux_params=auxParams,optimizer=opt,num_epoch=90)

In [ ]:
model.fit(X=trainIter, eval_data=valIter, eval_metric=metrics)